<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# 모듈 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import time 

In [7]:

# 검색 결과가 표시되는 iframe으로 전환
def search_iframe():
    driver.switch_to.default_content() # 차상위 (기본) 프레임으로 전환
    driver.switch_to.frame("searchIframe") # 검색 결과 프레임으로 전환

def entry_iframe():
    driver.switch_to.default_content() # 차상위 (기본) 프레임으로 전환
    WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="entryIframe"]'))) # WebDriverWait : entryIframe이 나타날 때까지 대기

    # for 문으로 총 5번까지 시도, 0.5초간 대기 후 프레임 전환을 시도 -> frame 전환에 성공하면 break 문으로 빠져나온다
    for i in range(5):
        time.sleep(1.5)
        try:
            driver.switch_to.frame(driver.find_element(By.XPATH, '//*[@id="entryIframe"]'))
            break
        except:
            pass

# 화면에 표시된 업체명을 확인하는 함수
def chk_names():
    search_iframe() # search 프레임으로 전환
    elem = driver.find_elements(By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/a[1]/div/div/span[1]') # 해당 프레임에서 Xpath를 사용하여 업체명이 표시된 요소를 찾음 원래는 //*[@id="_pcmap_list_scroll_container"]/ul/li/div[1]/div/a[1]/div/div/span[1]
    name_list = [e.text for e in elem] # find_elements 함수로 업체명이 표시된 요소를 모두 찾고, 그 중 text 값을 가져와서 리스트에 저장

    return elem, name_list


def chk_details():
    entry_iframe() # 세부 정보의 iframe으로 전환
    soup = BeautifulSoup(driver.page_source, 'html.parser')
#     print(soup)
    # 데이터 쌓기 - 업종, 주소, url 순
    try:
        category = soup.find('span', {'class': 'lnJFt'})
    except:
        category = float('nan')


    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        ID = float('nan')

    search_iframe() # 다시 검색 결과의 iframe으로 전환

    return category, url, ID

def crawling_main():
    global naver_res # 전역변수 naver_res 사용 (크롤링 데이터를 저장하는 데이터프레임)
    category_list = [] # 업종 리스트
    url_list = [] # url 리스트
    ID_list = []

    for e in elem:
        try:
            e.click()
            category, url, ID = chk_details() # 업체의 세부 정보 가져오기
        except:
            category = ''
            url=''
            ID =''
        

        # 가져온 정보를 리스트에 추가
        category_list.append(category)
        url_list.append(url)
        ID_list.append(ID)

    naver_temp = pd.DataFrame([name_list, category_list, url_list, ID_list], index = naver_res.columns).T
    naver_res = pd.concat([naver_res, naver_temp])
    naver_res.to_excel('./naver_crawling_result.xlsx')



# webdriver 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
keyword = '서대문구 창천동 맛집'
url = f'https://map.naver.com/p/search/{keyword}'
driver.get(url)
# 마우스 및 키보드 동작 시뮬레이터
action = ActionChains(driver)
# 불러올 정보를 담을 빈 데이터프레임 생성
naver_res = pd.DataFrame(columns=['업체명','업종','URL','ID'])

last_name = ''



page_num = 1

while True:
    time.sleep(1.5)
    search_iframe()
    elem, name_list = chk_names()
    if last_name == name_list[-1]:
        break
    while True:
        # 자동 스크롤 구현부
        driver.execute_script("arguments[0].scrollIntoView(true);", elem[-1])
        time.sleep(1.5)
        elem, name_list = chk_names()
        if last_name == name_list[-1]:
            break
        else:
            last_name = name_list[-1]
    crawling_main()
    # 다음 페이지로 이동
    try:
        next_button = driver.find_elements(By.CLASS_NAME, 'eUTV2')[-1]
        next_button.click()
    except:
        break


In [8]:
naver_res

,업체명,업종,URL,ID
0,오적회관 신촌점,[한식],https://pcmap.place.naver.com/restaurant/14910...,1491059869
1,아마 신촌본점,[인도음식],https://pcmap.place.naver.com/restaurant/13027...,13027492
2,정육면체 신촌점,[아시아음식],https://pcmap.place.naver.com/restaurant/12292...,1229208931
3,한술식당 신촌점,[한식],https://pcmap.place.naver.com/restaurant/13208...,1320893582
4,돈불1971,[돼지고기구이],https://pcmap.place.naver.com/restaurant/36343...,36343564
...,...,...,...,...
65,바플라이 1호점 : 별밤,[요리주점],https://pcmap.place.naver.com/restaurant/11609...,11609575
66,김사부 브런치&지엔빙,[중식당],https://pcmap.place.naver.com/restaurant/12853...,1285314962
67,만냥하우스,[한식],https://pcmap.place.naver.com/restaurant/10705...,1070522601
68,평화김해뒷고기 신촌점,"[육류,고기요리]",https://pcmap.place.naver.com/restaurant/19857...,1985788052


In [9]:
set(naver_res['업종'].apply(lambda x: x.text))

{'게요리',
 '고기뷔페',
 '곰탕,설렁탕',
 '곱창,막창,양',
 '국밥',
 '국수',
 '김밥',
 '낙지요리',
 '냉면',
 '다이어트,샐러드',
 '닭갈비',
 '닭발',
 '닭볶음탕',
 '닭요리',
 '대게요리',
 '덮밥',
 '돈가스',
 '돼지고기구이',
 '딤섬,중식만두',
 '떡볶이',
 '마라탕',
 '만두',
 '맥주,호프',
 '멕시코,남미음식',
 '바(BAR)',
 '백반,가정식',
 '백숙,삼계탕',
 '베이커리',
 '베트남음식',
 '복어요리',
 '브런치',
 '빙수',
 '생선회',
 '샤브샤브',
 '소고기구이',
 '순대,순댓국',
 '술집',
 '스테이크,립',
 '스파게티,파스타전문',
 '아시아음식',
 '양꼬치',
 '양식',
 '오므라이스',
 '요리주점',
 '우동,소바',
 '육류,고기요리',
 '이자카야',
 '이탈리아음식',
 '인도음식',
 '일본식라면',
 '일식당',
 '전통,민속주점',
 '조개요리',
 '족발,보쌈',
 '종합분식',
 '죽',
 '중식당',
 '찜닭',
 '초밥,롤',
 '치킨,닭강정',
 '카레',
 '카페',
 '카페,디저트',
 '칼국수,만두',
 '케이크전문',
 '태국음식',
 '토스트',
 '패밀리레스토랑',
 '피자',
 '한식',
 '해물,생선요리',
 '햄버거',
 '향토음식',
 '홍차전문점',
 '후렌치후라이'}

In [10]:
len(set(naver_res['업종'].apply(lambda x: x.text)))

75